# Faraday

This tutorial explains how Faraday works and how to use Faraday to train a generative model and generate synthetic smart meter data.

---

Faraday is a Conditional Variational Auto-Encoder (VAE)-based model. Unlike traditional VAEs where seeds are drawn from a normal distribution and decoded, Faraday works by:
1. First train a VAE using the following loss functions: a. MMD instead of KL-divergence b. Quantile losses and c. Mean squared error
2. Encode real samples to the latent space using the encoder, and fit a Gaussian Mixture Model (GMM) over the latent space.
3. During inference, draw samples from the GMM and decode with the decoder.

For more information on Faraday's architecture, refer to the [Faraday paper](https://arxiv.org/abs/2404.04314).


### Pre-requisites

If you haven't already, please download LCL dataset from [data.london.gov.uk](https://data.london.gov.uk/dataset/smartmeter-energy-use-data-in-london-households). 


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

import sys

import logging

logger = logging.getLogger(__name__)

# 💿 Loading LCL Data

In [3]:
from pathlib import Path
from opensynth.data_modules.lcl_data_module import LCLDataModule
import pytorch_lightning as pl

import matplotlib.pyplot as plt

data_path = Path("../../data/processed/historical/train/lcl_data.csv")
stats_path = Path("../../data/processed/historical/train/mean_std.csv")
outlier_path = Path("../../data/processed/historical/train/outliers.csv")

dm = LCLDataModule(data_path=data_path, stats_path=stats_path, batch_size=25000, n_samples=100000)
dm.setup()

In [4]:
dm_with_outliers = LCLDataModule(data_path=data_path, stats_path=stats_path, batch_size=200, n_samples=20000, outlier_path=outlier_path)
dm_with_outliers.setup()

# 🤖 VAE Module

In [ ]:
from opensynth.models.faraday.vae_model import FaradayVAE
# Option to pass in your own encoder architecture in the future
model = FaradayVAE(class_dim=2, latent_dim=16, learning_rate=0.001, mse_weight=3)

In [ ]:
# Batch size 500 is when MPS becomes faster than CPU..
# But sometimes large batch size hurts convergence..
# Suggest training on CPU with small batch size
# And potentially experiment with best hyperparameters on large batch size before using 'mps'

trainer = pl.Trainer(max_epochs=150, accelerator="auto")
trainer.fit(model, dm_with_outliers)

In [ ]:
import torch
torch.save(model, "vae_model.pt")

# 🕸️ GMM Module

In [5]:
from opensynth.models.faraday.model import FaradayModel
import numpy as np
import torch

In [6]:
# torch.save(model, "faraday_model.pt")
model = torch.load("vae_model.pt")

Need to update model s.t. `feature_list` is saved if want to load model from checkpoint. For now, use trained VAE. 

In [ ]:
# model = FaradayVAE.load_from_checkpoint("lightning_logs/version_0/checkpoints/epoch=249-step=25000.ckpt", map_location=torch.device('cpu'))

# faraday_model_1500 = FaradayModel(vae_module=model, n_components=1500, max_iter=100, tol=1e-2)
# faraday_model_150 = FaradayModel(vae_module=model, n_components=150, max_iter=100, tol=1e-2)
# faraday_model_50 = FaradayModel(vae_module=model, n_components=50, max_iter=100, tol=1e-2)
# faraday_model_10 = FaradayModel(vae_module=model, n_components=10, max_iter=100, tol=1e-2)
# faraday_model_1 = FaradayModel(vae_module=model, n_components=1, max_iter=100, tol=1e-2)


In [21]:
faraday_model_1500 = FaradayModel(
    vae_module=model, n_components=1500, tol=1e-2, gmm_max_epochs=100,  gmm_covariance_reg=1e-4,
)

faraday_model_150 = FaradayModel(
    vae_module=model, n_components=150, tol=1e-2, gmm_max_epochs=100, gmm_covariance_reg=1e-4,
)

faraday_model_50 = FaradayModel(
    vae_module=model, n_components=50, tol=1e-2, gmm_max_epochs=100, gmm_covariance_reg=1e-4,
)


faraday_model_10 = FaradayModel(
    vae_module=model, n_components=10, tol=1e-2, gmm_max_epochs=100, gmm_covariance_reg=1e-4,
)

faraday_model_1 = FaradayModel(
    vae_module=model, n_components=1, tol=1e-2, gmm_max_epochs=100, gmm_covariance_reg=1e-4,
)

In [24]:
gmm_data_module = LCLDataModule(data_path=data_path, stats_path=stats_path, batch_size=25000, n_samples=100000, outlier_path=outlier_path)
gmm_data_module.setup()

In [25]:
faraday_model_10.train_gmm(dm=gmm_data_module)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/charlotte.avery/.virtualenvs/OpenSynth-BNsxhSIM/lib/python3.11/site-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
/Users/charlotte.avery/.virtualenvs/OpenSynth-BNsxhSIM/lib/python3.11/site-packages/pytorch_lightning/core/optimizer.py:182: `LightningModule.configure_optimizers` returned `None`, this fit will run with no optimizer

  | Name                      | Type                    | Params | Mode 
------------------------------------------------------------------------------
0 | gmm_module                | GaussianMixtureModel    | 0      | train
1 | vae_module                | FaradayVAE              | 402 K  | train
2 | weight_metric             | WeightsMetric           | 0      | train
3 | mean_metric               | MeansMetric             | 0      | train
4 | precision_ch

tensor(0.6399)
tensor(1.)
Initial prec chol: 0.6398732662200928.                 Initial mean: 1.5001500844955444
Epoch 0: 100%|██████████| 4/4 [00:01<00:00,  2.40it/s, v_num=120]Local weights at rank: 0 - means: 0.0136, 0.7431
Reduced weights, means, covar: 0.0136,0.7431, 2.2767
NLL:  tensor(5.6046)
Epoch 1: 100%|██████████| 4/4 [00:01<00:00,  2.61it/s, v_num=120]Local weights at rank: 0 - means: 0.0235, 0.8159
Reduced weights, means, covar: 0.0235,0.8159, 1.5115
NLL:  tensor(4.8463)
Epoch 2: 100%|██████████| 4/4 [00:01<00:00,  2.49it/s, v_num=120]Local weights at rank: 0 - means: 0.0307, 0.8763
Reduced weights, means, covar: 0.0307,0.8763, 1.1191
NLL:  tensor(4.7586)
Epoch 3: 100%|██████████| 4/4 [00:01<00:00,  2.50it/s, v_num=120]Local weights at rank: 0 - means: 0.0379, 0.8284
Reduced weights, means, covar: 0.0379,0.8284, 0.8263
NLL:  tensor(4.6384)
Epoch 4: 100%|██████████| 4/4 [00:01<00:00,  2.66it/s, v_num=120]Local weights at rank: 0 - means: 0.0474, 0.7392
Reduced weights, mea

In [40]:
faraday_model_50.train_gmm(dm=gmm_data_module)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/charlotte.avery/.virtualenvs/OpenSynth-BNsxhSIM/lib/python3.11/site-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
/Users/charlotte.avery/.virtualenvs/OpenSynth-BNsxhSIM/lib/python3.11/site-packages/pytorch_lightning/core/optimizer.py:182: `LightningModule.configure_optimizers` returned `None`, this fit will run with no optimizer

  | Name                      | Type                    | Params | Mode 
------------------------------------------------------------------------------
0 | gmm_module                | GaussianMixtureModel    | 0      | train
1 | vae_module                | FaradayVAE              | 402 K  | train
2 | weight_metric             | WeightsMetric           | 0      | train
3 | mean_metric               | MeansMetric             | 0      | train
4 | precision_ch

Initial prec chol: 1.5353962182998657.                 Initial mean: 0.4759025275707245
Epoch 0: 100%|██████████| 4/4 [00:01<00:00,  2.09it/s, v_num=129]Local weights at rank: 0 - means: 0.0091, 0.3047
Reduced weights, means: 0.0091,0.3047, 
NLL:  tensor(4.1499)
Epoch 1: 100%|██████████| 4/4 [00:01<00:00,  2.09it/s, v_num=129]Local weights at rank: 0 - means: 0.0134, 0.1913
Reduced weights, means: 0.0134,0.1913, 
NLL:  tensor(3.6442)
Epoch 2: 100%|██████████| 4/4 [00:02<00:00,  1.90it/s, v_num=129]Local weights at rank: 0 - means: 0.0162, 0.1454
Reduced weights, means: 0.0162,0.1454, 
NLL:  tensor(3.4114)
Epoch 3: 100%|██████████| 4/4 [00:02<00:00,  1.88it/s, v_num=129]Local weights at rank: 0 - means: 0.0187, 0.0777
Reduced weights, means: 0.0187,0.0777, 
NLL:  tensor(3.3159)
Epoch 4: 100%|██████████| 4/4 [00:01<00:00,  2.05it/s, v_num=129]Local weights at rank: 0 - means: 0.0211, 0.0088
Reduced weights, means: 0.0211,0.0088, 
NLL:  tensor(3.1954)
Epoch 5: 100%|██████████| 4/4 [00:02<

In [27]:
faraday_model_150.train_gmm(dm=gmm_data_module)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/charlotte.avery/.virtualenvs/OpenSynth-BNsxhSIM/lib/python3.11/site-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
/Users/charlotte.avery/.virtualenvs/OpenSynth-BNsxhSIM/lib/python3.11/site-packages/pytorch_lightning/core/optimizer.py:182: `LightningModule.configure_optimizers` returned `None`, this fit will run with no optimizer

  | Name                      | Type                    | Params | Mode 
------------------------------------------------------------------------------
0 | gmm_module                | GaussianMixtureModel    | 0      | train
1 | vae_module                | FaradayVAE              | 402 K  | train
2 | weight_metric             | WeightsMetric           | 0      | train
3 | mean_metric               | MeansMetric             | 0      | train
4 | precision_ch

tensor(2.2308)
tensor(1.0000)
Initial prec chol: 2.2307753562927246.                 Initial mean: 1.3672646284103394
Epoch 0: 100%|██████████| 4/4 [00:03<00:00,  1.14it/s, v_num=122]Local weights at rank: 0 - means: 0.0017, 1.0868
Reduced weights, means, covar: 0.0017,1.0868, 0.1874
NLL:  tensor(3.4614)
Epoch 1: 100%|██████████| 4/4 [00:02<00:00,  1.43it/s, v_num=122]Local weights at rank: 0 - means: 0.0018, 0.8566
Reduced weights, means, covar: 0.0018,0.8566, 0.1473
NLL:  tensor(2.9998)
Epoch 2: 100%|██████████| 4/4 [00:03<00:00,  1.32it/s, v_num=122]Local weights at rank: 0 - means: 0.0023, 0.8240
Reduced weights, means, covar: 0.0023,0.8240, 0.0956
NLL:  tensor(2.7432)
Epoch 3: 100%|██████████| 4/4 [00:03<00:00,  1.33it/s, v_num=122]Local weights at rank: 0 - means: 0.0030, 0.7130
Reduced weights, means, covar: 0.0030,0.7130, 0.1017
NLL:  tensor(2.6538)
Epoch 4: 100%|██████████| 4/4 [00:02<00:00,  1.34it/s, v_num=122]Local weights at rank: 0 - means: 0.0049, 0.7192
Reduced weights,

In [28]:
ligthning_sum_components = faraday_model_150.gmm_module.means.sum(axis=1)
len(ligthning_sum_components[ligthning_sum_components==0])

69

In [43]:
faraday_model_1500.train_gmm(dm=gmm_data_module)

Epoch 0:   0%|          | 0/4 [14:50<?, ?it/s]


GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/charlotte.avery/.virtualenvs/OpenSynth-BNsxhSIM/lib/python3.11/site-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
/Users/charlotte.avery/.virtualenvs/OpenSynth-BNsxhSIM/lib/python3.11/site-packages/pytorch_lightning/core/optimizer.py:182: `LightningModule.configure_optimizers` returned `None`, this fit will run with no optimizer

  | Name                      | Type                    | Params | Mode 
------------------------------------------------------------------------------
0 | gmm_module                | GaussianMixtureModel    | 0      | train
1 | vae_module                | FaradayVAE              | 402 K  | train
2 | weight_metric             | WeightsMetric           | 0      | train
3 | mean_metric               | MeansMetric             | 0      | train
4 | precision_ch

Initial prec chol: 5.030717849731445.                 Initial mean: 1.7637825012207031
Epoch 10: 100%|██████████| 4/4 [00:16<00:00,  0.24it/s, v_num=131]


In [ ]:
# torch.save(faraday_model_1500, "faraday_model_1500.pt")
# torch.save(faraday_model_150, "faraday_model_150.pt")
# torch.save(faraday_model_50, "faraday_model_50.pt")
# torch.save(faraday_model_10, "faraday_model_10.pt")
# torch.save(faraday_model_1, "faraday_model_1.pt")

# 📈 Comparing Results

### 0. Generating Samples

In [44]:
def generate_synthetic_samples(model, n_samples, dm):
    gmm_samples = model.sample_gmm(n_samples)
    gmm_samples_reconstructed = dm.reconstruct_kwh(gmm_samples['kwh'])
    gmm_samples_reconstructed = torch.clip(gmm_samples_reconstructed, min=0)
    return gmm_samples_reconstructed

In [45]:
gmm_1500 = generate_synthetic_samples(faraday_model_1500, 20000, dm)
gmm_150 = generate_synthetic_samples(faraday_model_150, 20000, dm)
gmm_50 = generate_synthetic_samples(faraday_model_50, 20000, dm)
gmm_10 = generate_synthetic_samples(faraday_model_10, n_samples=20000, dm=dm)
gmm_1 = generate_synthetic_samples(faraday_model_1, 20000, dm)

AttributeError: 'GaussianMixtureModel' object has no attribute 'model'

In [ ]:
real_kwh = dm.reconstruct_kwh(next(iter(gmm_data_module.train_dataloader()))['kwh'])
real_kwh = torch.clip(real_kwh, min=0) # Clip min 0 to get read of negative values

### 1. Comparing mean, 95th quantile, median profiles

In [ ]:
def plot_stats(real_kwh, gmm_reconstruct):

    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 3), sharey=True, gridspec_kw={'wspace': 0.05})

    ax1.plot(real_kwh.mean(dim=0).detach().numpy(), label="real kwh")
    ax1.plot(gmm_reconstruct.mean(dim=0).detach().numpy(), label="gmm kwh")
    ax1.set_title("Mean kWh per half hour")
    ax1.set_xlabel("Settlement Periods")
    ax1.legend()

    ax2.plot(real_kwh.quantile(0.95, dim=0).detach().numpy(), label="real kwh")
    ax2.plot(gmm_reconstruct.quantile(0.95, dim=0).detach().numpy(), label="gmm kwh")
    ax2.set_title("95th Quantile kWh per half hour")
    ax2.set_xlabel("Settlement Periods")
    ax2.legend()

    ax3.plot(real_kwh.quantile(0.5, dim=0).detach().numpy(), label="real kwh")
    ax3.set_title("Median kWh per half hour")
    ax3.set_xlabel("Settlement Periods")
    ax3.plot(gmm_reconstruct.quantile(0.5, dim=0).detach().numpy(), label="gmm kwh")
    ax3.legend()

    fig.text(0.1, 0.5, 'kWh', va='center', rotation='vertical')

In [ ]:
plot_stats(real_kwh, gmm_1500)

In [ ]:
plot_stats(real_kwh, gmm_150)

In [ ]:
plot_stats(real_kwh, gmm_50)

In [ ]:
plot_stats(real_kwh, gmm_10)

In [ ]:
plot_stats(real_kwh, gmm_1)

### 2. PCA and TSNE Distribution Plots

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import numpy as np


def train_pca_and_tsne(real_kwh, gmm_reconstruct):
    pca = PCA(n_components=2)
    tsne = TSNE(n_components=2)

    pca.fit(real_kwh.detach().numpy())
    pca_real = pca.transform(real_kwh.detach().numpy())
    pca_gmm = pca.transform(gmm_reconstruct.detach().numpy())

    tsne_input = np.concatenate([real_kwh.detach().numpy(), gmm_reconstruct.detach().numpy()])
    tsne_results = tsne.fit_transform(tsne_input)
    tsne_real = tsne_results[:len(real_kwh)]
    tsne_gmm = tsne_results[len(real_kwh):]

    return pca_real, pca_gmm, tsne_real, tsne_gmm


def plot_pca_tsne(pca_real, pca_gmm, tsne_real, tsne_gmm):
    fig, (ax_pca, ax_tsne) = plt.subplots(1, 2, figsize=(12, 4))

    ax_pca.scatter(pca_real[:, 0], pca_real[:, 1], label="real kwh", s=0.3, alpha=0.5)
    ax_pca.scatter(pca_gmm[:, 0], pca_gmm[:, 1], label="gmm kwh", s=0.3, alpha=0.5)
    ax_pca.set_title("PCA")
    ax_pca.set_xlabel("PCA 1")
    ax_pca.set_ylabel("PCA 2")
    ax_pca.legend()

    ax_tsne.scatter(tsne_real[:, 0], tsne_real[:, 1], label="real kwh", s=0.3, alpha=0.5)
    ax_tsne.scatter(tsne_gmm[:, 0], tsne_gmm[:, 1], label="gmm kwh", s=0.3, alpha=0.5)
    ax_tsne.set_title("TSNE")
    ax_tsne.set_xlabel("TSNE 1")
    ax_tsne.set_ylabel("TSNE 2")
    ax_tsne.legend()
    return fig


In [ ]:
a1, a2, a3, a4 = train_pca_and_tsne(real_kwh, gmm_1500)
_ = plot_pca_tsne(a1, a2, a3, a4)
plt.title("GMM with 1500 clusters")

In [ ]:
b1, b2, b3, b4 = train_pca_and_tsne(real_kwh, gmm_150)
_ = plot_pca_tsne(b1, b2, b3, b4)
plt.title("GMM with 150 clusters")

In [ ]:
c1, c2, c3, c4 = train_pca_and_tsne(real_kwh, gmm_50)
_ = plot_pca_tsne(c1, c2, c3, c4)
plt.title("GMM with 50 clusters")

In [ ]:
d1, d2, d3, d4 = train_pca_and_tsne(real_kwh, gmm_10)
_ = plot_pca_tsne(d1, d2, d3, d4)
plt.title("GMM with 10 clusters")

In [ ]:
e1, e2, e3, e4 = train_pca_and_tsne(real_kwh, gmm_1)
_ = plot_pca_tsne(e1, e2, e3, e4)
plt.title("GMM with 1 clusters")

# 🛃 Customising your VAE Architecture

It's possible to also customise your VAE archicture without touching the rest of Faraday code. You can do this by:

1. Creating a custom class inheriting from the Encoder and Decoder module
2. Using `super().__init__()` to inherit all the attributes and methods of the parent class
3. Overriding the attribute `encoder_layers` or `decoder_layers`. 

In this example, we'll be showing how to do this with simple linear layers, but in reality you could use more complicated architectures such as Conv1D layers, or LSTM layers.
For more complex layers, you'll need to make sure that you've shaped the inputs correctly.

In [ ]:
from opensynth.models.faraday.vae_model import FaradayVAE, Encoder, Decoder
from opensynth.models.faraday.model import FaradayModel
import torch.nn as nn
import torch

### Create Custom Encoder and Decoder Architectures.

In [ ]:
class CustomEncoderModule(Encoder):
    """
    Custom Encoder Module
    """
    def __init__(self, latent_dim: int, input_dim: int, class_dim: int):
        """
        Inherit parent encoder attributes and methods.
        But we will be overriding the `encoder_layers` attribute
        with our custom encoder architecture.

        When inheriting from parent `Encoder` class, we need to
        pass in the attributes: latent_dim, input_dim, class_dim.

        Outputs of encoder_layers should be `latent_dim`.

        Args:
            latent_dim (int): Latent dimension.
            input_dim (int): Input dimensions.
            class_dim (int): Class dimensions.
        """
        super().__init__(latent_dim=latent_dim, input_dim=input_dim, class_dim=class_dim)
        self.encoder_layers = nn.Sequential(
            nn.Linear(self.encoder_input_dim, 1024),
            nn.GELU(),
            nn.Linear(1024, self.latent_dim)
        )


class CustomDecoderModule(Decoder):
    """
    Custom Decoder Module
    """
    def __init__(self, class_dim: int, latent_dim: int, output_dim: int):
        """
        Inherit parent decoder attributes and methods.
        But we will be overriding the `decoder_layers` attribute
        with our custom decoder architecture.

        When inheriting from parent `Decoder` class, we need to
        pass in the attributes: class_dim, latent_dim, output_dim.

        Outputs of encoder_layers should be `output_dim`.

        Args:
            latent_dim (int): Latent dimension.
            output_dim (int): Output dimensions.
            class_dim (int): Class dimensions.
        """
        super().__init__(class_dim=class_dim, latent_dim=latent_dim, output_dim=output_dim)
        self.decoder_layers = nn.Sequential(
            nn.Linear(self.latent_dim, 1024),
            nn.GELU(),
            nn.Linear(1024, self.output_dim)
        )

### Initiate Faraday VAE with custom encoder and decoders

In [ ]:
custom_encoder = CustomEncoderModule(class_dim=2, latent_dim=16, input_dim=48)
custom_decoder = CustomDecoderModule(class_dim=2, latent_dim=16, output_dim=48)

faraday_custom_vae = FaradayVAE(
    class_dim=2, 
    latent_dim=16, 
    learning_rate=0.001, 
    mse_weight=3, 
    custom_encoder=custom_encoder, 
    custom_decoder=custom_decoder
)

custom_trainer = pl.Trainer(max_epochs=250, accelerator="cpu")
custom_trainer.fit(faraday_custom_vae, dm)


In [ ]:
faraday_custom_model = FaradayModel(vae_module=faraday_custom_vae, n_components=15, tol=1e-5, gmm_covariance_reg=1e-3)
gmm_data_module = LCLDataModule(data_path=data_path, stats_path=stats_path, batch_size=5000, n_samples=50000)
gmm_data_module.setup()
faraday_custom_model.train_gmm(dm=gmm_data_module)

### Check Results

In [ ]:
custom_gmm_samples = faraday_custom_model.sample_gmm(n_samples=10000)

custom_gmm_kwh = custom_gmm_samples['kwh']
custom_gmm_reconstruct = dm.reconstruct_kwh(custom_gmm_kwh)
custom_gmm_reconstruct = torch.clip(custom_gmm_reconstruct, min=0) # Clip min 0 to get read of negative values

In [ ]:
custom_real_kwh = dm.reconstruct_kwh(next(iter(gmm_data_module.train_dataloader()))['kwh'])
custom_real_kwh = torch.clip(custom_real_kwh, min=0) # Clip min 0 to get read of negative values

In [ ]:
plot_stats(custom_real_kwh, custom_gmm_reconstruct)

In [ ]:
a,b,c,d = train_pca_and_tsne(custom_real_kwh, custom_gmm_reconstruct)

In [ ]:
_ = plot_pca_tsne(a, b, c, d)

# 🔐 Training with Differential Privacy (DP-SGD)

Privacy should be one of the concerns with genarating synthetic data, especially when done for data sharing purposes. To implement privacy, we use Differentially-Private Stochastic Gradient Descent [[1]](https://arxiv.org/abs/1607.00133) implemented with Pytorch Opacus library [[2]](https://opacus.ai/).

You can train Faraday with Differential Privacy turned on with the `differential_privacy` parameter as demonstrated below. When implementing DP-SGD, you need to specify the following:

1. `epsilon` - the level of privacy (high epsilon = less private)
2. `delta` - this should be $<<1/N$, where $N$ is the size of the dataset

For more context and considerations between Privacy and synthetic smart meter generation, e.g. on deciding `epsilon` value, check out this paper from Centre for Net Zero: [Defining "Good": Evalution Framework for SYnthetic Smart Meter Data](https://arxiv.org/abs/2407.11785).

In [ ]:
from opensynth.models.faraday.vae_model import FaradayVAE

vae_dp = FaradayVAE(
    class_dim=2,
    latent_dim=16,
    learning_rate=0.001,
    mse_weight=3,
    differential_privacy=True,
    epsilon=8.0,
    delta=1/20000,
)

dp_trainer = pl.Trainer(max_epochs=25, accelerator="cpu")
dp_trainer.fit(vae_dp, dm)